load packages

In [1]:
#standard packages
import numpy as np
from numpy import concatenate
import pandas as pd
import glob
import math
import warnings

#plot packages
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)

#package to create date-time objects
import datetime as dt

#performance measures
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from timeit import default_timer as timer
from datetime import timedelta
import tensorflow as tf
from tqdm import tqdm
from IPython.utils import io

import warnings
warnings.filterwarnings("ignore")

def _naive_forecasting(actual: np.ndarray, seasonality):
    return actual[:-seasonality]

def mean_absolute_scaled_error(actual: np.ndarray, predicted: np.ndarray, seasonality):
    #Baseline (benchmark) is computed with naive forecasting shifted bu seasonality
    return mean_absolute_error(actual, predicted) / mean_absolute_error(actual[seasonality:], _naive_forecasting(actual, seasonality))

def mean_directional_accuracy(y, y_hat):
    #Mean Directional Accuracy
    return np.mean(np.sign(y.diff()[1:]) == np.sign(y_hat.diff()[1:]))
    # the reason the two diffrence are diffrentely computed is because y is inputed as dataframe column but yhat is a array. 
    #If this is diffrent then change.
    
#RF packages
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
from sklearn import tree

pd.options.mode.chained_assignment = None  # default='warn'
# import random 
import random

#prophet packages
from prophet import Prophet

#LSTM packages
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

#SARIMAX packages
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm                       
import warnings
from itertools import product
from tqdm import tqdm_notebook

import seaborn as sns

First we load the dataset, then we transform the date variable to date-time formate and change the column names of the date and target variable to ds and y. Then we drop the AvgSerice variable as we do not which to use this.

In [2]:
df = pd.read_csv(r'C:\Users\Britt\Documents\DataAnalyse\data\half_hour.txt', sep="\t")
df.date = pd.date_range(df.date[0] , periods = len(df.date), freq='30min')
df.rename(columns={'date' : 'ds', 'count' : 'y'}, inplace=True)
df.drop(['AvgService'], axis=1, inplace=True)

For m=48 this model is unable to run do to memory problems, therefore we need to aggregate the data to hourly to decrease the seasonality cycle length.

To resample the data we first have to make the data-time variable the index on which is resampled.

In [9]:
df['ds'] = pd.to_datetime(df['ds'])
df.set_index('ds', inplace=True)

Then we create a new dataframe that has the same columns as df except for kperiod, this is the half-hour index and is thus irrelevant in the new set and is thus droped.

In [13]:
dfSARIMAX=pd.DataFrame(columns=df.columns)
dfSARIMAX=dfSARIMAX.drop('kperiod', axis = 1)
dfSARIMAX['y']=df.resample('H').sum().y
dfSARIMAX[['irregular', 'holiday', 'DeliverD1', 'DeliverD7',
       'DeliverD14', 'DeliverD21', 'Bill1', 'Bill7', 'Bill14', 'Bill21',
       'Deliver']]=df.resample('H').last()[['irregular', 'holiday', 'DeliverD1', 'DeliverD7',
       'DeliverD14', 'DeliverD21', 'Bill1', 'Bill7', 'Bill14', 'Bill21',
       'Deliver']]

In [15]:
dfSARIMAX.head()

,irregular,holiday,y,DeliverD1,DeliverD7,DeliverD14,DeliverD21,Bill1,Bill7,Bill14,Bill21,Deliver
ds,,,,,,,,,,,,
2004-02-14 00:00:00,0,0,118,0,0,0,0,0,0,0,0,0
2004-02-14 01:00:00,0,0,70,0,0,0,0,0,0,0,0,0
2004-02-14 02:00:00,0,0,39,0,0,0,0,0,0,0,0,0
2004-02-14 03:00:00,0,0,42,0,0,0,0,0,0,0,0,0
2004-02-14 04:00:00,0,0,16,0,0,0,0,0,0,0,0,0


# SARIMAX

First, we set the len_trainset to 20% and split up the data into train and test.

In [20]:
len_trainset=round(0.2*len(dfSARIMAX))
train = dfSARIMAX[:len_trainset]
test = dfSARIMAX[len_trainset:]

The gird search of the model is preformed by the function auto_arima

In [23]:
sxmodel = pm.auto_arima(train[['y']], exog = train[['irregular', 'holiday', 'DeliverD1', 'DeliverD7', 'DeliverD14', 'DeliverD21', 'Bill1', 'Bill7', 'Bill14', 'Bill21','Deliver']],
                       start_p=4, start_q=1,
                       start_P=6, start_Q=0,
                        max_p=6, max_q=6,
                        max_P=6, max_Q=6,
                        max_order=6,
                       test='adf',
                       m=24,seasonal=True,
                       d=None, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True, 
                       stepwise=True, max_iter=20)

sxmodel.summary()

Performing stepwise search to minimize aic


MemoryError: Unable to allocate 349. MiB for an array with shape (172, 172, 1547) and data type float64

For the in-sample prediction we use the function fittedvalues() note that for the first seasonal cycle these will be off as it uses the values of the previous -24 time steps and for the first cycle these dont exsist and are set to 0. For the computation of the performance these should be removed.

In [ ]:
pred = sxmodel.fittedvalues()

We use the forecast function to forcast the future

In [ ]:
forecast = sxmodel.predict(n_periods=len(test), exog=test[['irregular', 'holiday', 'DeliverD1', 'DeliverD7',
       'DeliverD14', 'DeliverD21', 'Bill1', 'Bill7', 'Bill14', 'Bill21',
       'Deliver']])

Then we put them together

In [ ]:
prediction = pd.concat([pred, forecast])

# Results

In [ ]:
y_true_train = df['y'][:len_trainset] #actual values of y for training set
y_true_test = df['y'][len_trainset:]  #actual values of y for test set
y_pred_train = prediction[:len_trainset] #predicted values of y for training set
y_pred_test = prediction[len_trainset:]  #predicted values of y for test set

MAEtrain = mean_absolute_error(y_true_train, y_pred_train) #in-sample MAE
MAEtest = mean_absolute_error(y_true_test, y_pred_test)    #out-of-sample MAE

print('train MAE', MAEtrain)
print('test MAE', MAEtest)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True, constrained_layout=True, figsize=(13, 3))

ax.set_title('SARIMAX')
ax.plot(np.asarray(results_arima.ds, dtype='datetime64[s]'), results_arima.y, color= '#555B6E', label = 'Actual')
# Plot the predicted values
ax.plot(np.asarray(results_arima.ds, dtype='datetime64[s]'), results_arima.Forecast, label = 'Prediction', alpha=0.9, color= '#F3B768')
ax.axvline(x=np.asarray(results_arima.ds, dtype='datetime64[s]')[len_trainset], color='k', linestyle='--', alpha=0.7)
ax.margins(x=0.01)
ax.margins(y=0.02)
ax.legend(loc="upper left")

fig.supxlabel('Date')
fig.supylabel('Number of calls')

In [ ]:
zoom1 = 50 #amount of observation from the train set you see
zoom2 = 400  #amount of observation from test you see
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True,  constrained_layout=True, figsize=(13, 3))

ax.set_title('SARIMAX')
ax.plot(np.asarray(results_arima.ds, dtype='datetime64[s]')[len_trainset-zoom1:len_trainset+zoom2], results_arima.y[len_trainset-zoom1:len_trainset+zoom2], color= '#555B6E', label = 'Actual')
# Plot the predicted values
ax.plot(np.asarray(results_arima.ds, dtype='datetime64[s]')[len_trainset-zoom1:len_trainset+zoom2], results_arima.Forecast[len_trainset-zoom1:len_trainset+zoom2], label = 'Prediction', alpha=0.9, color= '#F3B768')
ax.axvline(x=np.asarray(results_arima.ds, dtype='datetime64[s]')[len_trainset], color='k', linestyle='--', alpha=0.7)
ax.margins(x=0.01)
ax.margins(y=0.02)
ax.legend(loc="upper left")
fig.supxlabel('Date')
fig.supylabel('Number of calls')